## Recreate our Galactic Models FITS, this time as one FITS file (aka model)

Going to do all of this a second time but using the raw data * wavelength^2 in order to get accurate Fnu shape


In [1]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import astropy.constants as const
import astropy.units as u
import io 

In [2]:
files = []
for file in os.listdir('../data/galaxySEDs/'):
    if file.endswith('.norm_1p6'):
        files.append(file)

## Create FITS

In [3]:
from astropy.io import fits
from astropy import table
from astropy import units as u
from astropy import constants as const

In [6]:
tab = [] # Tab is a list of dataframes that contain the SEDs for each template
for file in files:
    tab.append(pd.read_csv('../data/galaxySEDs/'+file,header = None, delim_whitespace=True))
modnames = [file.split('.')[0] for file in files]
a1 = np.ones(len(tab)) # Validity array
primhead = fits.Header([fits.Card('DISTANCE',3E+21,'Distance in cm'),
                    fits.Card('NWAV',len(tab[0][0]),'Number of wavelengths')])
primhdu = fits.PrimaryHDU(a1,primhead)

col1 = fits.Column(name='MODEL_NAME', format='20A', array=modnames) #Column objects package the arrays with the typical fits info: name and format
cols = fits.ColDefs([col1]) #ColDefs object packages the column objects
namehdu = fits.BinTableHDU.from_columns(cols,name = 'MODEL_NAMES') #Creates the bintablehdu object to be made to FITS_rec

wavlist = (tab[0][0]/10**4).values
freqlist = const.c.value / wavlist
speccol1 = fits.Column(name='WAVELENGTH',format = 'D', unit = 'um', array=wavlist)
speccol2 = fits.Column(name='FREQUENCY', format = 'D', unit = 'Hz', array=freqlist)
speccols = fits.ColDefs([speccol1,speccol2])
spechdu = fits.BinTableHDU.from_columns(speccols,name = 'SPECTRAL_INFO')

# Not sure if apertures matter or should be the same as for YSOs? I'm thinking that we have 0 apertures because we only have data
# for one model, not across apertures or 10000 models.
#aplist = np.logspace(2,6,20)
#apcol1 = fits.Column(name='APERTURE', format = 'D', unit = 'AU', array=aplist)
#apcols = fits.ColDefs([apcol1])
#aphdu = fits.BinTableHDU.from_columns(apcols,name = 'APERTURES')

# Giving synthetic unit here: mJy, despite not being in mJy, normalized at 1.6 um, index 6396
valdata = np.array([(tab[i][1]*(wavlist**2)) / tab[i][1][6396] for i in range(len(tab))])
valdata = valdata.reshape((32,1,11005))
valhdr = fits.Header([fits.Card('BUNIT','mJy','')])
valhdu = fits.ImageHDU(data=valdata,name='VALUES',header = valhdr)

# Uncertainty in the models?? Don't have any anymore
uncdata = np.zeros((32,1,11005))
unchdr = fits.Header([fits.Card('BUNIT','mJy','')])
unchdu = fits.ImageHDU(data=uncdata,name='UNCERTAINTIES',header = unchdr)

hdulist = fits.HDUList([primhdu, namehdu, spechdu, valhdu, unchdu])
hdulist.writeto('../data/galaxtemps2/flux.fits',overwrite=False)


# Create models.conf file #
with open('../data/galaxtemps2/models.conf','w') as modcon:
    l = ['name = galaxtemps2\n', 'length_subdir = 2\n', 'aperture_dependent = no\n', 'logd_step = 0.02\n', 'version = 2\n']
    modcon.writelines(l)

## Create parameters.FITS (Only a list of model names for us)

In [7]:
for i in range(0,len(files)):
   modnames = [file.split('.')[0] for file in files]
   parmhead = fits.Header()
   parmhdu = fits.PrimaryHDU(parmhead)
   parmcol = fits.Column(name='MODEL_NAME', format='20A', array=modnames)
   parmcols = fits.ColDefs([parmcol]) 
   pnamehdu = fits.BinTableHDU.from_columns(parmcols,name = 'MODEL_NAMES')
   parmhdulist = fits.HDUList([parmhdu, pnamehdu])
   parmhdulist.writeto('../data/galaxtemps2/parameters.fits',overwrite=True)

In [8]:
with fits.open('../data/galaxtemps2/parameters.fits') as testfits:
    testfits.info()
    #print(repr(testfits[3].header))
    #print(repr(testfits[3].data))
    print(repr(testfits[1].header))
    print(repr(testfits[1].data))

Filename: ../data/galaxtemps2/parameters.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   (0,)      
  1  MODEL_NAMES    1 BinTableHDU     11   32R x 1C   [20A]   
XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   20 / length of dimension 1                          
NAXIS2  =                   32 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    1 / number of table fields                         
TTYPE1  = 'MODEL_NAME'                                                          
TFORM1  = '20A     '                           

## Convolve our new model directory with the 8 filters using our saved responses in data/filterResponses

In [9]:
from sedfitter.filter import Filter
from sedfitter.convolve import convolve_model_dir

In [10]:
#List of filter names: 2J = 1.235, 2H = 1.662, 2K = 2.159, I1 = 3.55, I2 = 4.493,  I3 = 5.8, I4 = 8.0, M1 = 24.0 
filternames = ['2J', '2H', '2K', 'I1', 'I2', 'I3', 'I4', 'M1']
filterpeaks = [1.235,1.662,2.159,3.6,4.5,5.8,8.0,23.675]

In [11]:
def clean_2MASS_resp(filtname):
    restab = pd.read_csv('../data/filterResponses/'+filtname,header=None,delim_whitespace=True)
    length = restab.shape[0]
    restab = restab.drop([0,1,2,length-5,length-4,length-3,length-2,length-1],axis=0)
    restab = restab.drop([2,3,4,5],axis=1)
    restab = restab.astype(float)
    restab.index = range(len(restab))
    return restab

def clean_IRAC_resp(filename):
    restab = pd.read_csv('../data/filterResponses/'+filename,header=None,delim_whitespace=True)
    restab = restab.drop([0,1,2],axis=0)
    restab = restab.drop([2,3,4,5,6,7,8,9],axis=1)
    restab = restab.astype(float)
    restab.index = range(len(restab))
    return restab

def clean_MIPS_resp(filename):
    restab = pd.read_csv('../data/filterResponses/'+filename,header=None,delim_whitespace=True)
    restab = restab.drop([2,3,4,5,6],axis=1)
    restab = restab.drop([0,1,2],axis=0)
    restab = restab.iloc[0:128]
    restab = restab.astype(float)
    restab.index = range(len(restab))
    return restab

In [12]:
twoJResp = clean_2MASS_resp('2J')
twoHResp = clean_2MASS_resp('2H')
twoKResp = clean_2MASS_resp('2K')
Ione = clean_IRAC_resp('I1')
Itwo = clean_IRAC_resp('I2')
Ithree = clean_IRAC_resp('I3')
Ifour = clean_IRAC_resp('I4')
Mone = clean_MIPS_resp('M1')

responses = [twoJResp, twoHResp, twoKResp, Ione, Itwo, Ithree, Ifour, Mone]

responses[0]

,0,1
0,1.062,0.000000
1,1.066,0.000407
2,1.070,0.001543
3,1.075,0.002670
4,1.078,0.005506
...,...,...
102,1.416,0.008648
103,1.421,0.000729
104,1.426,0.000348
105,1.442,0.000378


In [13]:
filters = []
for i in range(len(filternames)):
    f = Filter()
    f.name = filternames[i]
    f.central_wavelength = filterpeaks[i] * u.micron
    f.nu = (list(reversed(responses[i][0].values)) * u.micron).to(u.Hz,equivalencies = u.spectral())
    f.response = list(reversed(responses[i][1].values))
    f.normalize()
    filters.append(f)

In [14]:
convolve_model_dir('../data/galaxtemps2',filters)

INFO: 32 SEDs found in ../data/galaxtemps2 [sedfitter.convolve.convolve]


In [15]:
with fits.open('../data/galaxtemps2/convolved/M1.fits') as convfits:
    convfits.info()
    print(repr(convfits[0].header))
    print(repr(convfits[0].data))
    print(repr(convfits[1].header))
    print(repr(convfits[1].data))

Filename: ../data/galaxtemps2/convolved/M1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1  CONVOLVED FLUXES    1 BinTableHDU     19   32R x 3C   [30A, D, D]   
SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
FILTWAV =               23.675                                                  
NMODELS =                   32                                                  
NAP     =                    1                                                  
None
XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 